In [168]:
import numpy as np
import pandas as pd
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [169]:
veri = {'Outlook':          ["Sunny", "Sunny", "Overcast", "Rain", "Rain", "Rain", "Overcast", "Sunny", "Sunny", "Rain", "Sunny", "Overcast", "Overcast", "Rain"],
        'Temperature':      ["Hot","Hot","Hot","Mild","Cool","Cool","Cool","Mild","Cool","Mild","Mild","Mild","Hot","Mild"],
        'Humidity':         ["High", "High","High","High","Normal","Normal","Normal", "High", "Normal","Normal","Normal","High","Normal","High"],
        'Wind':             ["Weak", "Strong", "Weak","Weak","Weak","Strong","Strong","Weak","Weak","Weak","Strong","Strong","Weak","Strong"],
        'PlayedFootball':   ["No","No", "Yes", "Yes", "Yes", "No", "Yes","No", "Yes","Yes", "Yes","Yes","Yes","No"]}

In [170]:
df = pd.DataFrame(veri)
df

,Outlook,Temperature,Humidity,Wind,PlayedFootball
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [171]:
# Benzersiz verilerin sütun bazında tespitinin yapılması:
sutunOzellikSayimlari = {}
for i in range(len(df.columns)):
    sayim = df[df.columns[i]].value_counts()
    sutunOzellikSayimlari[df.columns[i]] = sayim.to_dict()
    print(f'{df.columns[i]} sutunundaki verilerin listesi: {sayim}')
    print('\n\n')

Outlook sutunundaki verilerin listesi: Outlook
Sunny       5
Rain        5
Overcast    4
Name: count, dtype: int64



Temperature sutunundaki verilerin listesi: Temperature
Mild    6
Hot     4
Cool    4
Name: count, dtype: int64



Humidity sutunundaki verilerin listesi: Humidity
High      7
Normal    7
Name: count, dtype: int64



Wind sutunundaki verilerin listesi: Wind
Weak      8
Strong    6
Name: count, dtype: int64



PlayedFootball sutunundaki verilerin listesi: PlayedFootball
Yes    9
No     5
Name: count, dtype: int64





In [172]:
sutunOzellikSayimlari

{'Outlook': {'Sunny': 5, 'Rain': 5, 'Overcast': 4},
 'Temperature': {'Mild': 6, 'Hot': 4, 'Cool': 4},
 'Humidity': {'High': 7, 'Normal': 7},
 'Wind': {'Weak': 8, 'Strong': 6},
 'PlayedFootball': {'Yes': 9, 'No': 5}}

In [173]:
def sozluk_cevir(sozluk):
    # Bu fonksiyon, iç içe sözlükleri derinlikli listeye dönüştürür.
    sonuc_listesi = []  # Sonuç için boş bir liste oluşturulur.
    for anahtar, deger in sozluk.items():  # Sözlüğün anahtar ve değer çiftleri üzerinde döngü yapılır.
        icerik = [anahtar]  # Anahtar, sonuç listesine eklenir.
        if isinstance(deger, dict):  # Eğer değer yine bir sözlükse,
            icerik.extend(sozluk_cevir(deger))  # Fonksiyon kendini tekrar çağırarak iç içe sözlükleri çözer. Fonksiyon sonucunu doğrudan ekleyin, bir liste içine almayın.
        else:
            icerik.append(deger)  # Değer bir sözlük değilse doğrudan eklenir.
        sonuc_listesi.append(icerik)  # Oluşturulan içerik, sonuç listesine eklenir.
    return sonuc_listesi  # Sonuç listesi döndürülür.

cevrilmis_ifade = sozluk_cevir(sutunOzellikSayimlari)
cevrilmis_ifade

[['Outlook', ['Sunny', 5], ['Rain', 5], ['Overcast', 4]],
 ['Temperature', ['Mild', 6], ['Hot', 4], ['Cool', 4]],
 ['Humidity', ['High', 7], ['Normal', 7]],
 ['Wind', ['Weak', 8], ['Strong', 6]],
 ['PlayedFootball', ['Yes', 9], ['No', 5]]]

In [174]:
# Satır elemanlarının sayımı:
# Ana fonksiyon, verilen iç içe listelerden sayısal değerleri çıkarır
def sayim_listesi(veri_listesi):
    tum_sayimlar = []  # Sonuçları saklamak için bir liste

    # Sayıları çıkarmak için yardımcı özyinelemeli fonksiyon
    def sayilari_cikar(liste):
        sayilar = []  # Sayıları saklamak için liste
        for oge in liste:  # Her öğeyi kontrol et
            if isinstance(oge, list):  # Eğer öğe bir liste ise
                sayilar.extend([sayilari_cikar(oge)])  # Özyineleme ile içindeki sayıları ekle
            elif isinstance(oge, int or float):  # Eğer öğe bir sayı ise
                sayilar.append(oge)  # Sayıyı listeye ekle
        return sayilar  # Sayılar listesini döndür

    for item in veri_listesi:  # Ana liste içindeki her öğeyi kontrol et
        sayilar = sayilari_cikar(item[1:])  # İlk öğe hariç diğerlerinden sayıları çıkar
        tum_sayimlar.append(sayilar)  # Bulunan sayıları sonuç listesine ekle
    
    return tum_sayimlar  # Sonuç listesini döndür

# Fonksiyonu çağır ve sonucu yazdır
tum_sayim_listesi = sayim_listesi(cevrilmis_ifade)
tum_sayim_listesi

[[[5], [5], [4]], [[6], [4], [4]], [[7], [7]], [[8], [6]], [[9], [5]]]

In [175]:
# Her verikümesi için ayrı ayrı olasılıkları hesapla
def olasilik(olasilikListe):
    olasilikSonuc = []
    for alt_liste in olasilikListe:
        toplam = 0
        for eleman in alt_liste:
            toplam += eleman[0]  # Her bir elemanın ilk değerini al
        alt_olasiliklar = [i[0] / toplam for i in alt_liste]  # Her bir elemanın olasılığını hesapla
        olasilikSonuc.append(alt_olasiliklar)  # Oluşan olasılıkları sonuç listesine ekle
    return olasilikSonuc

veri = [[[5], [5], [4]], [[6], [4], [4]], [[7], [7]], [[8], [6]], [[9], [5]]]
tumOlasilik = olasilik(veri)
print(tumOlasilik)

[[0.35714285714285715, 0.35714285714285715, 0.2857142857142857], [0.42857142857142855, 0.2857142857142857, 0.2857142857142857], [0.5, 0.5], [0.5714285714285714, 0.42857142857142855], [0.6428571428571429, 0.35714285714285715]]


In [176]:
# Olasılıkları ayır
etiketOlasilik_y = tumOlasilik[-1]
ozelliklerOlasilik_x = tumOlasilik[:-1]
ozelliklerOlasilik_x

[[0.35714285714285715, 0.35714285714285715, 0.2857142857142857],
 [0.42857142857142855, 0.2857142857142857, 0.2857142857142857],
 [0.5, 0.5],
 [0.5714285714285714, 0.42857142857142855]]

In [177]:
# Entropileri hesapla
def entropy(liste):
    toplam = 0
    for eleman in liste:
        if isinstance(eleman, (int, float)):
            islem = (-1) * (eleman)  * math.log(eleman ,2)
            toplam += islem
        if isinstance(eleman, list):
            for alt_eleman in eleman:
                if isinstance(alt_eleman, (int, float)):
                    islem = (-1) * (alt_eleman)  * math.log(alt_eleman ,2)
                    toplam += islem
                else:
                    print("Listenin derinlik sayısı fazla, işlenemeyen eleman:", alt_eleman)
        else:
            print("Hata: İşlenemeyen eleman:", eleman)
    return toplam
entropi_y = entropy(etiketOlasilik_y)
entropi_y


Hata: İşlenemeyen eleman: 0.6428571428571429
Hata: İşlenemeyen eleman: 0.35714285714285715


0.9402859586706309

In [178]:
entropi_y

0.9402859586706309

In [180]:
# Kök değer bulma hesabı:
# Entropi(Outlook) = Entropi(5,5,4)
# Entropi(Temperature) = Entropi(6,4,4)
# Entropi(Humidity) = Entropi(7,7)
# Entropi(Wind) = Entropi(8,6)

# Gain(S,A) = Entropy(S) - sigma((|Sv/S|)*Entropy(Sv))
"""
Gain(S,A) --> Bilgi kazancı
Entropy(S) --> Etiketin entropisi
Entropy(Sv) --> Özelliğin entropisi
"""
# Algoritma:
"""
0- Verisetinin entropisini hesaplayan bir fonksiyon oluştur ve bunu listeye ekle. Verisetinin entropisi aslında y'nin yani sonucun(PlayedFootball) etiketinin entropisidir. Entropy(S)=Entropy(PlayedFootball)
1- her sütun için benzersiz veri sayıları bulunan sözlüğü oluştur. A = {indeks-1: 3, indeks-2: 5, ....}; B= {indeks-1: 4, indeks-2: 8, ....}
2- her sütun indeksi için olasılıkları hesaplamak için fonksiyon oluştur.
3- her sütun için entropiyi hesaplayan fonksiyonu oluştur.
4- bulunan her entropi değerini bir liste/sözlük içerisine ekle.
5- her sütun için bilgi kazancını hesaplayan fonksiyonu oluştur.
6- her sözlük bir sütunu ifade eder. Tüm sözlüklerin entropisini hesaplayıp bir sözlük içerisine ekle
"""

"\n0- Verisetinin entropisini hesaplayan bir fonksiyon oluştur ve bunu listeye ekle. Verisetinin entropisi aslında y'nin yani sonucun(PlayedFootball) etiketinin entropisidir. Entropy(S)=Entropy(PlayedFootball)\n1- her sütun için benzersiz veri sayıları bulunan sözlüğü oluştur. A = {indeks-1: 3, indeks-2: 5, ....}; B= {indeks-1: 4, indeks-2: 8, ....}\n2- her sütun indeksi için olasılıkları hesaplamak için fonksiyon oluştur.\n3- her sütun için entropiyi hesaplayan fonksiyonu oluştur.\n4- bulunan her entropi değerini bir liste/sözlük içerisine ekle.\n5- her sütun için bilgi kazancını hesaplayan fonksiyonu oluştur.\n6- her sözlük bir sütunu ifade eder. Tüm sözlüklerin entropisini hesaplayıp bir sözlük içerisine ekle\n"